In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
import pickle
with open('data', 'rb') as f:
    data = pickle.load(f)
for key in data:
    print(key)

In [ ]:
print(data['happy'].shape)

In [4]:
# print(np.max(data['happy']))
# print(np.min(data['happy']))
# divy=np.max(data['happy'])-np.min(data['happy'])
# y=(data['happy']-np.min(data['happy']))/divy
# print(np.max(y),(np.min(y)))

In [5]:
# print(np.max(data['neutral']))
# print(np.min(data['neutral']))
# divx=np.max(data['neutral'])-np.min(data['neutral'])
# x=(data['neutral']-np.min(data['neutral']))/divx
# print(np.max(x),(np.min(x)))

x = data['neutral']
y = data['happy']
z = data['sad']

x_train = y
#X_train = np.concatenate((x[:350],x[:350]))

print(x_train.shape)
#print(X_train.shape)

(400, 25800)


In [6]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

encoding_dim = 1024

input_img = Input(shape=(25800,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(25800)(encoded)

autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


In [7]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [8]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')#from keras.datasets import mnist

In [43]:
#x = data['happy']
#y = data['happy']
##z = data['sad']

#x_train = x[:350]
#X_train = y[:350]
autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=32,
                shuffle=True,
                validation_data=(y[350:375],y[350:375])
               )# encode and decode some digits
# note that we take them from the *test* set
# encoded_imgs = encoder.predict(x_test)
# decoded_imgs = decoder.predict(encoded_imgs)

Train on 400 samples, validate on 25 samples
Epoch 1/20
400/400 [==============================] - 12s - loss: 46.0355 - val_loss: 42.3917
Epoch 2/20
400/400 [==============================] - 11s - loss: 46.8344 - val_loss: 44.1400
Epoch 3/20
400/400 [==============================] - 11s - loss: 48.0676 - val_loss: 41.5719
Epoch 4/20
400/400 [==============================] - 11s - loss: 46.4390 - val_loss: 41.7176
Epoch 5/20
400/400 [==============================] - 11s - loss: 47.1516 - val_loss: 41.2369
Epoch 6/20
400/400 [==============================] - 11s - loss: 44.8013 - val_loss: 40.7748
Epoch 7/20
400/400 [==============================] - 11s - loss: 45.3187 - val_loss: 39.8647
Epoch 8/20
400/400 [==============================] - 11s - loss: 43.8747 - val_loss: 39.8190
Epoch 9/20
400/400 [==============================] - 11s - loss: 43.9652 - val_loss: 39.8003
Epoch 10/20
400/400 [==============================] - 11s - loss: 43.3747 - val_loss: 40.3121
Epoch 11/20
40

In [44]:
xy = data['neutral'][380].reshape((1, 25800))
# print(x[380].shape)
# print(xy.shape)
encoded_imgs = encoder.predict(xy)
mfccs = decoder.predict(encoded_imgs)

# mfccs = mfccs.reshape((200, 25800//200))
print(mfccs)
#print(divy)
#print(np.min(data['happy']))
print(mfccs.shape)

[[ -4.90356384e+02  -1.93518867e+01   2.45771580e+01 ...,  -1.68877527e-01
    5.39366484e-01  -9.62687194e-01]]
(1, 25800)


In [45]:
import librosa
# import numpy as np
from IPython.lib.display import Audio

def invlogamplitude(S):
    """librosa.logamplitude is actually 10_log10, so invert that."""
    return 10.0**(S/10.0)

# filename = "a.wav"
# y, sr = librosa.load(filename)

# Y = librosa.stft(y)
# mfccs = librosa.feature.mfcc(y, n_mfcc=100)
def recon(mfccs):
    sr = 22050

    n_mfcc = mfccs.shape[0]
    n_mel = 128
    dctm = librosa.filters.dct(n_mfcc, n_mel)
    n_fft = 2048
    mel_basis = librosa.filters.mel(sr, n_fft)

    bin_scaling = 1.0/np.maximum(0.0005, np.sum(np.dot(mel_basis.T, mel_basis),
    axis=0))

    recon_stft = bin_scaling[:, np.newaxis] * np.dot(mel_basis.T,
    invlogamplitude(np.dot(dctm.T, mfccs)))

    excitation = np.random.randn(mfccs.shape[1] * 510)
    E = librosa.stft(excitation)
    recon = librosa.istft(E/np.abs(E)*np.sqrt(recon_stft))

    print(mfccs.shape)
    return Audio(recon,rate=sr)


In [46]:
recon(data['happy'][380].reshape((200, 129), order='F'))

(200, 129)


In [47]:
mxx = data['neutral'][380].reshape((200, 129), order='F')
# mxx = mxx * divx
# mxx += np.min(data['neutral'])

recon(mxx)
# recon(mfccs)

(200, 129)


In [48]:
myy = mfccs.reshape((200, 129), order='F')
# myy = myy * divy
# myy += np.min(data['happy'])

recon(myy*2)

(200, 129)


In [42]:
model_json= autoencoder.to_json()
with open("autoencoder.json","w") as json_file:
    json_file.write(model_json)
    
autoencoder.save_weights("autoencoder.h5py")

ImportError: `save_weights` requires h5py.